In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from pandas.core.groupby.generic import ScalarResult
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPRegressor
import warnings
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# 1.0 for clean catch, 0.7 for clean field, 0.5 for bobble, 0 for muff, None for Touchback, OOB, etc
contact_mappings = {'CC': 1.0, 'CFFG': 0.9, 'BB': 0.72, 'BC': 0.57, 'BF': 0.77, 'BOG': 0.52, 'ICC': 0.25, 'MBC': 0.0, 'MBDR': 0.0}
pd.set_option('display.max_columns', None)

# Modeling Expected Punt Return Yards
Although perhaps not as glamorous as the offensive phase or as schematically complicated as the defensive phase, the punt is one of the most pivotal plays in the game of football.  Bridging the gap between the two main phases of the game, a good punt can set a defense up for success by pinning the opposing team deep in their own territory, shrinking the playbook down to low-risk, low-reward plays and increasing the odds of getting the ball back.  Conversely, a good punt return can set an offense up for success.  The game of football is a balance of scheme and execution, technique and athleticism.  What makes a punt and corresponding return so special is that in no other phase of the game does that balance shift more towards instinct and natural ability.  While there are certain basic strategies a unit may take, the play essentially boils down to a runner and 10 blockers versus 11 defenders trying to tackle the runner.  This lack of diversity in strategy has made quantifying and projecting punt/punt return performance traditionally difficult.  However, with the tracking data provided by the competition, we will attempt to build a new metric, **Expected Punt Return Yards**, to evaluate returner performance using *Punt Return Yards Over Expected* (**PRYOE**), and conversely to evaluate punt coverage performance using *Punt Return Yards Over Expected Allowed* (**PRYOEA**).  

In [ ]:
def read_inputs():
    ## Read input data
    scouting_df = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
    games_df = pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
    plays_df = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
    tracking_df = pd.DataFrame()
    tracking_2018 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2018.csv')
    tracking_2019 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2019.csv')
    tracking_2020 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')
    tracking_df = tracking_df.append(tracking_2018)
    tracking_df = tracking_df.append(tracking_2019)
    tracking_df = tracking_df.append(tracking_2020)
    players_df = pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
    return scouting_df, games_df, plays_df, tracking_df, players_df
scouting_df, games_df, plays_df, tracking_df, players_df = read_inputs()

In [ ]:
def distanceBetween(x1, y1, x2, y2):
    a = np.array((x1, y1))
    b = np.array((x2, y2))
    c = np.linalg.norm(a-b)
    return c

def getFrameIDOfPuntCatch(tracking_df, plays_df):
    plays_df['season'] = plays_df.apply(lambda x: int(str(x['gameId'])[:4]), axis=1)
    punts_df = plays_df[['gameId', 'playId', 'season', 'specialTeamsPlayType', 'returnerId']].query('specialTeamsPlayType == "Punt"')
    temp_df = tracking_df[['event', 'gameId', 'playId', 'frameId']]
    temp_df = temp_df.query('event == "kick_received" | event == "punt_land" | event == "punt_downed" | event == "punt_received" | event == "punt_muffed" | event == "fair_catch"')
    temp_df.set_index(['gameId', 'playId'], inplace=True)
    punts_df.set_index(['gameId', 'playId'], inplace=True)
    catch_df = temp_df.join(punts_df, on=['gameId', 'playId'], how='inner')
    catch_df = catch_df.drop_duplicates()
    catch_df.head()
    return catch_df

def getDistancesAtTimeOfCatch(catch_df, tracking_df):
    catch_df.reset_index(inplace=True)
    
    catch_df.dropna(subset=['returnerId'], axis=0, inplace=True)
    
    tracking_df.set_index(['gameId', 'playId', 'frameId'], inplace=True)
    catch_df.set_index(['gameId', 'playId', 'frameId'], inplace=True)
    
    full_df = catch_df.join(tracking_df, on=['gameId', 'playId', 'frameId'], how='inner', rsuffix='r_')
    
    full_df.reset_index(inplace=True)
    catch_df.reset_index(inplace=True)
    
    punts_returners = catch_df[['gameId', 'playId', 'returnerId']].values.tolist()
    
    col_arr = []
    yd_arr = []
    in_radius_arr = []
    for punt in punts_returners:
        play = full_df.query('gameId == @punt[0] & playId == @punt[1]').dropna(subset=['nflId'])
        returner_id = int(punt[2].split(';')[0])
        returner_loc = play.query('nflId == @returner_id')[['x','y', 'team']].values.tolist()
        closest_loc = 1000.0
        play_in_rad = 0
        for row in play.itertuples():
            if int(row.nflId) == returner_id or row.team == returner_loc[0][2]:
                pass
            else:
                this_loc = distanceBetween(returner_loc[0][0], returner_loc[0][1], row.x, row.y)
                if this_loc < closest_loc:
                    closest_loc = this_loc
                if this_loc < 8.0:
                    play_in_rad += 1
        col_arr.append(closest_loc)
        yd_arr.append(returner_loc[0][1])
        in_radius_arr.append(play_in_rad)
        
    col = pd.Series(data=col_arr)
    yd = pd.Series(data=yd_arr)
    in_rad = pd.Series(data=in_radius_arr)
    yd = yd.apply(lambda x: 0 if (x < 10.0 or x > 110.0) else (int(120.0 - x) if x > 60.0 else int(x)))
    catch_df['closest_gunner'] = col
    catch_df['catch_yardline'] = yd
    catch_df['defenders_within_radius'] = in_rad
    catch_df.drop(['frameId', 'season', 'specialTeamsPlayType', 'returnerId'], axis=1, inplace=True)
    
    return catch_df

def getReturnerAvg(df):
    df = df.copy()[['returnerId', 'kickReturnYardage']]
    returner_df = df.groupby(['returnerId']).mean()
    returner_df['returnAvg'] = returner_df['kickReturnYardage']
    returner_df.drop(['kickReturnYardage'], axis=1, inplace=True)
    return returner_df

# Feature Extraction

In order to build a successful model, we first need to consider which aspects of a play we can measure, and which may be impactful to a punt return.  

## Dataset Features
After examining the dataset, there are a number of measurements that jump out as immediately applicable.  These include:
* *Kick Length*
* *Punt Hangtime*
* *Number of Gunners (Punt team players that sprint down the field to tackle returner)*
* *Number of Punt Safeties (Punt team players that hang back)*
* *Number of Vises (Return team players that line up opposite gunners and prevent them from tackling returner)*
* *Number of Punt Rushers (Return team players that rush the punter and attempt to block the punt)*

## Encoded Features
There are also several columns that, given some encoding, can be easily quantified.  These include:
* *Kick Direction (intended and actual)*
    * This is encoded as the difference between the two.  E.g. (intended left, actual left) = 1.0, (intended left, actual center) = 0.5, (intended left, actual right) = 0.0.
* *Return Direction (intended and actual)*
    * Same scale as above.
* *Catch Quality*
    * Encoded on a range scale: 1.0 for clean catch, 0.7 for clean field, 0.5 for bobble, 0 for muff.
* *Snap Quality*
    * Binary encoding, 1 if snap quality was good ('OK' in dataset), and 0 if it was anything else. 

## Derived Features
Finally, using the tracking data, we can build some more complex features that give us insight into what happens during the play.  These include:
* *Yardline where the punt is received*
    * From perspective of leaving your own endzone.
* *Returner's past average return yards*
* *How close the nearest punt team player is to returner at time of catch*
* *Number of punt team players within returners **Radius of Concern** at time of catch*
    * For our purposes, **RoC** set to 8 yards.  With more time, this number could be optimized further.

## Truth Features
In order to model punt return yards, we need to identify both that a punt has been returned and how many yards were gained on the return.  Therefore our *Ground Truth* features are:
* *Was the punt returned?*
    * 1 for returned, 0 for not returned.
* *How many yards were gained on the return?*
    * Difference between yardline of catch and yardline of tackle.

In the case of our model, we are not concerned with non-routine play outcomes.  Events like blocked punts, fumbled snaps, fake punts, etc. are not returnable plays, and are therefore irrelevant to our investigation.  These plays have been removed from our feature set.  

In [ ]:
def encode_columns(plays_df, scouting_df, tracking_df, games_df):
    ## Preprocessing step for encoding inputs
    games_df = games_df.copy()
    games_df = games_df.reset_index(drop=True)
    
    # Get playIDs for all punts, as well as other raw columns that require no further manipulation
    params_df = plays_df[['gameId', 'playId', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'kickLength', 'kickReturnYardage', 'possessionTeam']].query('specialTeamsPlayType == "Punt"')
    scouting_df = scouting_df[['gameId', 'playId', 'snapDetail', 'operationTime', 'hangTime', 'kickType', 'kickDirectionIntended', 'kickDirectionActual', 'returnDirectionIntended', 'returnDirectionActual', 'gunners', 'puntRushers', 'specialTeamsSafeties', 'vises', 'kickContactType']]
    params_df.set_index(['gameId', 'playId'], inplace=True)
    scouting_df.set_index(['gameId', 'playId'], inplace=True)
    params_df = params_df.join(scouting_df, on=['gameId', 'playId'])
    games_df = games_df.reset_index()
    params_df = params_df.reset_index()
    
    # Non-inline Columns
    catch_df = getFrameIDOfPuntCatch(tracking_df, plays_df)
    temp_df = getDistancesAtTimeOfCatch(catch_df.copy(), tracking_df)
    temp_df.set_index(['gameId', 'playId'], inplace=True)
    
    # Derived Columns
    params_df['season'] = params_df['gameId'].apply(lambda x: games_df.loc[x == games_df['gameId'], ['season']].values[0][0])
    params_df.set_index(['gameId', 'playId'], inplace=True)
    params_df['snapQuality'] = params_df['snapDetail'].apply(lambda x: 1 if x == 'OK' else 0)
    params_df['kickDirDiff'] = params_df[['kickDirectionIntended', 'kickDirectionActual']].apply(lambda x: 1 if x[0] == x['kickDirectionActual'] else (0.5 if (x['kickDirectionIntended'] == 'C' or x['kickDirectionActual'] == 'C') else 0), axis=1)
    params_df['retDirDiff'] = params_df[['returnDirectionIntended', 'returnDirectionActual']].apply(lambda x: 1 if x['returnDirectionIntended'] == x['returnDirectionActual'] else (0.5 if (x['returnDirectionIntended'] == 'C' or x['returnDirectionActual'] == 'C') else 0), axis=1)
    params_df['numGunners'] = params_df['gunners'].apply(lambda x: 0 if pd.isna(x) else (len(x.split(';'))))
    params_df['numVises'] = params_df['vises'].apply(lambda x: 0 if pd.isna(x) else (len(x.split(';'))))
    params_df['numSafeties'] = params_df['specialTeamsSafeties'].apply(lambda x: 0 if pd.isna(x) else (len(x.split(';'))))
    params_df['numRushers'] = params_df['puntRushers'].apply(lambda x: 0 if pd.isna(x) else (len(x.split(';'))))
    params_df['catchQuality'] = params_df['kickContactType'].apply(lambda x: contact_mappings[x] if x in contact_mappings.keys() else None)
    params_df['kickReturnYardage'].fillna(0, inplace=True)
    params_df['catchQuality'].fillna(0.0, inplace=True)
    returner_df = getReturnerAvg(params_df)
    params_df = params_df.join(returner_df, on=['returnerId'], how='left')
    params_df = params_df.join(temp_df, on=['gameId', 'playId'], how='inner')
    params_df['isReturn'] = params_df[['event', 'kickReturnYardage', 'catchQuality']].apply(lambda x: 0 if (x['event'] == 'fair_catch' or x['event'] == 'punt_downed' or (x['kickReturnYardage'] == 0 and x['event'] == 'punt_land' and x['catchQuality'] == 0)) else 1, axis=1)
    
    # Remove non-feature columns and re-index data
    params_df.drop(['gunners', 'vises', 'specialTeamsSafeties', 'puntRushers', 'returnDirectionIntended', 'returnDirectionActual', 'kickDirectionIntended', 'kickDirectionActual', 'snapDetail', 'kickType', 'kickContactType'], axis=1, inplace=True)
    params_df.reset_index(inplace=True)
    
    return params_df, returner_df
params_df, returners_df = encode_columns(plays_df, scouting_df, tracking_df, games_df)
params_df.head()

# Building a Model
Before attempting to mathematically model a punt return, we must first consider what goes through the punt returners head on a given play.  After the ball is punted, the punt returner has 3 options:
* Do nothing and let the ball land, allowing the punting team to "down" the ball wherever it may rest
* Catch the ball but not return it, either by a fair catch or taking a knee
* Catch the ball and return it

In the case of the first two outcomes, there is no return.  This allows us to reduce the initial step to a binary decision: *Should the punt be returned or not returned?*

From there, the next question that must be answered is: *Given that the punt is returned, how many yards do we expect the returner to gain?*

## Design
Now that we have identified the questions that must be answered, we can determine the optimal techniques for answering each.  

The first question is a yes/no question.  This lends itself to using a *Classification* technique.  Candidates for this include:
* Logistic Regression
* Ensemble Classification (e.g. Random Forest Classifier)
* Multilayer Perceptron (Neural Net) Binary Classifier

After testing these three candidate techniques, the most accurate model was using a Logistic Regression with Newton's Method as the specified solver function.  

The second question requires a quantity as an answer.  This lends itself to using a *Regression* technique.  Candidates for this include:
* Linear Regression
* Gradient Descent Regression
* Multilayer Perceptron (Neural Net) Regressor

After testing these model candidates, the most accurate turned out to be a variation of a Gradient Descent Regressor, a *Stochastic Gradient Descent* model.  *SGD* is essentially a Gradient Descent Regressor that uses a random subset of data to calculate error on each iteration instead of the entire dataset.  

## Testing
Our full, cleaned dataset includes about 3200 punts/returns from the 2018-2020 seasons.  We will reserve 200 plays for testing, and use the rest for training.

### Return Model
A typical Logistic Regression model returns a probability matrix, where the option with >50% probability is selected as the classification output.  However, when testing our model, it was discovered that moving that threshold to 60% probability resulted in significantly higher accuracy.  The breakdown is as follows:

In [ ]:
def test_logistic_model(df):
    df = df.copy()
    outcomes = ['True Positive', 'False Positive', 'True Negative', 'False Negative']
    t_p = 0
    f_p = 0
    t_n = 0
    f_n = 0
    df.drop(['gameId', 'playId', 'event', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'kickReturnYardage', 'possessionTeam', 'season'], axis=1, inplace=True)
    model = LogisticRegression(solver='newton-cg')
    df = df.sample(frac=1).reset_index(drop=True)
    y = df['isReturn'].to_numpy()
    X = df.drop(['isReturn'], axis=1)
    X.fillna(0.0, inplace=True)
    X = normalize(X, axis=0)
    Xone = X[:-200]
    yone = y[:-200]
    Xtwo = X[-199:]
    ytwo = y[-199:]

    model.fit(Xone, yone)
    total = len(ytwo)
    correct = 0
    for i in range(len(ytwo)):
        yres = model.predict([Xtwo[i]])
        confidence = model.predict_proba([Xtwo[i]])
        manual = 0
        if confidence[0][1] > 0.6 and ytwo[i] == 1:
            t_p += 1
        elif confidence[0][1] > 0.6 and ytwo[i] == 0:
            f_p += 1
        elif confidence[0][1] < 0.6 and ytwo[i] == 1:
            f_n += 1
        elif confidence[0][1] < 0.6 and ytwo[i] == 0:
            t_n += 1
        if confidence[0][1] > 0.6:
            manual = 1
        if manual == ytwo[i]:
            correct += 1
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'xy'}, {'type': 'domain'}]], subplot_titles=['Confusion Distribution', 'Accuracy Breakdown'])
    fig.add_trace(go.Bar(x = outcomes, y = [t_p, f_p, t_n, f_n], name="Specific Results", showlegend=False), 1,1)
    fig.layout['yaxis'].update(title_text="Count")
    fig.add_trace(go.Pie(labels = ['Correct', 'Incorrect'], values = [t_p + t_n, f_p + f_n], name="Overall Results"), 1,2)
    fig.show()
    manual_score = correct/total
    print("manual score: {}".format(manual_score))
test_logistic_model(params_df)

This model ends up giving us an accuracy of between 80% and 90%.  

## Yardage Model
With the yardage model, evaluation becomes slightly more difficult.  An error of 2 when the prediction is 2 yards and the actual is 4 yards seems to be a miss, whereas an error of 2 when the prediction is 25 yards and the actual is 27 yards seems to be fairly accurate.  To be consistent with scoring from our binary return model, we also need our score to be in the range [0,1] (inclusive).  To account for this, we will score using the equation: 

$$score = \dfrac{(1 - \left| \dfrac{\dfrac{y_{_{pred}}}{2} + y_{_{pred}}}{y_{act}} - \dfrac{\dfrac{y_{_{pred}}}{2}+y_{act}}{y_{act}} \right|) + 1}{\max(y_{{pred}}, y_{act})}$$

This may seem overly complex, but allows us to scale the error on a curve according to the magnitude of projection and truth, while accounting for both positive and negative predictions and actual values.  Given more time, a more concise scoring function could be designed, which would allow for more accurate evaluation.  

In [ ]:
def test_linear_model(df):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df = df.copy()
        df = df.query('isReturn == 1')
        df.drop(['gameId', 'playId', 'event', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'isReturn', 'possessionTeam', 'season'], axis=1, inplace=True)
        model = SGDRegressor(alpha=0.000002, penalty='elasticnet', max_iter=3000)

        y = df['kickReturnYardage'].to_numpy()
        X = df.drop(['kickReturnYardage'], axis=1)
        X.fillna(0.0, inplace=True)

        X = normalize(X, axis=0) 

        Xone = X[:-200]
        yone = y[:-200]
        Xtwo = X[-199:]
        ytwo = y[-199:]
        model.fit(Xone, yone)
        ypred = []
        yact = []
        yscore = []
        ycorr = 0
        yinc = 0
        for i in range(len(ytwo)):
            pred = model.predict([Xtwo[i]])[0]
            act = ytwo[i]
            if act == 0:
                act = 0.1
            yards_score = (1.0 - abs(((pred/2 + int(pred)) / pred) - ((pred/2 + int(act)) / pred))) + 1 / (max(pred, act))
            if yards_score < 0:
                yards_score = 0
            if yards_score > 0.8:
                ycorr += 1
            else:
                yinc += 1
            if yards_score > 1.0:
                yards_score=1
            ypred.append(pred)
            yact.append(act)
            yscore.append(yards_score)
        fig = go.Figure(go.Scatter(x=yact, y=ypred, name='Score', mode='markers', hovertemplate="<b>Pred. Yards</b>: %{y:.2f}<br><b>Act. Yards</b>: %{x:.2f}", marker=dict(color=yscore, colorscale='rdbu_r', showscale=True)))
        fig.layout['xaxis'].update(title_text='Actual Yards')
        fig.layout['yaxis'].update(title_text='Expected Yards')
        fig.update_layout(title_text="Actual Yards vs Expected Yards", title_x=0.5, title_xanchor='center', legend_title="Score")
        fig.show()
        score = model.score(Xtwo, ytwo)
        print(score)
        print('% in acceptable range: {}%'.format((ycorr / (ycorr + yinc)) * 100))
test_linear_model(params_df)

As you can see, the model struggles with returns at either extreme.  However, it is excellent at predicting yardage in the 3-10 yard range, which is where most "expected" returns are.  The outliers greater than 15 yards and less than 1 yard are examples of returns where a returner had significantly more or less **YoE**, respectively.  

In [ ]:
def fit_logistic_model(df):
    df = df.copy()
    df.drop(['gameId', 'playId', 'event', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'kickReturnYardage', 'possessionTeam', 'season'], axis=1, inplace=True)
    model = LogisticRegression(solver='newton-cg')
    df = df.sample(frac=1).reset_index(drop=True)
    y = df['isReturn'].to_numpy()
    X = df.drop(['isReturn'], axis=1)
    X.fillna(0.0, inplace=True)
    X = normalize(X, axis=0)
    model.fit(X, y)
    return model

def fit_linear_model(df):
    df = df.copy()
    df.drop(['gameId', 'playId', 'event', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'isReturn', 'possessionTeam', 'season'], axis=1, inplace=True)
    model = SGDRegressor(alpha=0.00002, penalty='elasticnet', max_iter=10000, early_stopping=True)
    
    y = df['kickReturnYardage'].to_numpy()
    X = df.drop(['kickReturnYardage'], axis=1)
    X.fillna(0.0, inplace=True)
    
    X = normalize(X, axis=0)    
        
    model.fit(X, y)
    return model

## Two-Step Model
Now that we have built and tested both of our models, it is time to put them together.  We will first pass each punt into our binary return model.  We can evaluate according to the following four scenarios:
* We predict *no return* and we get *no return*: We guessed correct, and can give ourselves a score of **1**
* We predict *return* and we get *no return*: We guessed wrong and can give ourselves a score of **0**
* We predict *no return* and we get *return*: We guessed wrong and can give ourselves a score of **0**
* We predict *return* and get *return*: We guessed correct, and will assign ourselves a score between **0 and 1** according to our above scoring function

In [ ]:
def linear_two_step(df):
    df = df.copy()
    scores = []
    df = df.sample(frac=1).reset_index(drop=True)
    train_df = df.iloc[:-300]
    test_df = df.iloc[-299:]
    return_model = fit_logistic_model(train_df)
    yardage_model = fit_linear_model(train_df)
    ytest = test_df[['isReturn', 'kickReturnYardage']].to_numpy()
    Xtest = test_df.drop(['gameId', 'playId', 'event', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'isReturn', 'kickReturnYardage', 'possessionTeam', 'season'], axis=1)
    Xtest.fillna(0.0, inplace=True)
    Xtest = normalize(Xtest, axis=0)
    linear_scores = []
    ypred = []
    yact = []
    yscore = []
    ycorr = 0
    yinc = 0
    for i in range(len(ytest)):
        return_res = return_model.predict_proba([Xtest[i]])
        return_manual = 0
        if return_res[0][1] > 0.6:
            return_manual = 1
        if return_manual == 0 and ytest[i][0] == 0:
            scores.append(1)
            ypred.append(0)
            yact.append(0)
            yscore.append(1)
            ycorr += 1
        elif return_manual == 1 and ytest[i][0] == 0:
            scores.append(0)
            ypred.append(1)
            yact.append(0)
            yscore.append(0)
            yinc += 1
        elif return_manual == 0 and ytest[i][0] == 1:
            scores.append(0)
            ypred.append(0)
            yact.append(1)
            yscore.append(0)
            yinc += 1
        else:
            yards_res = yardage_model.predict([Xtest[i]])[0]
            if ytest[i][1] == 0:
                ytest[i][1] = 0.1
            yards_score = (1.0 - abs(((yards_res/2 + int(yards_res)) / yards_res) - ((yards_res/2 + int(ytest[i][1])) / yards_res))) + 1 / (max(yards_res, ytest[i][1]))
            if yards_score < 0:
                yards_score = 0
            if yards_score > 0.8:
                ycorr += 1
            else:
                yinc +=1
            if yards_score > 1.0:
                yards_score=1
            scores.append(yards_score)
            linear_scores.append(yards_score)
            ypred.append(yards_res)
            yact.append(ytest[i][1])
            yscore.append(yards_score)
            
    linear_acc = sum(linear_scores) / len(linear_scores)
    overall_acc = sum(scores) / len(scores)
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'xy'}, {'type': 'domain'}]], subplot_titles=['Expected Return Yards vs Actual Return Yards', 'Accuracy Breakdown'])
    fig.add_trace(go.Scatter(x=yact, y=ypred, mode='markers', hovertemplate="<b>Pred. Yards</b>: %{y:.2f}<b><br>Act. Yards</b>: %{x:.2f}", marker=dict(color=yscore, colorscale='rdbu_r', showscale=False), showlegend=False))
    fig.layout['xaxis'].update(title_text='Actual Yards')
    fig.layout['yaxis'].update(title_text='Expected Yards')
    fig.add_trace(go.Pie(labels = ['Correct', 'Incorrect'], values = [ycorr, yinc], name="Overall Results"), 1,2)
    fig.show()
linear_two_step(params_df)

# Results
## Players
Now that we have established a model to predict Expected Punt Return Yards (EPRY), we can compare each players expected performance versus their actual performance.  If we sum a players EPRY and subtract that from the sum of their actual punt return yards.  This gives us **Punt Return Yards Over Expected** (**PRYOE**) for each player.  Of course, this is a cumulative value that depends on the number of opportunities for each player, so we can divide by the number of returns for each player, giving us an average **PRYOE** per return.


In [ ]:
def format_player_results(pryoe_df, players_df):
    pryoe_df = pryoe_df.copy()
    players_df = players_df.loc[:,['nflId', 'displayName']]
    players_df.loc[:,'returnerId'] = players_df['nflId']
    players_df = players_df.reset_index()
    players_df = players_df.drop(['nflId'], axis=1)
    pryoe_df.loc[:,'returnerId'] = pryoe_df['returnerId'].astype(int)
    players_df.loc[:,'returnerId'] = players_df['returnerId'].astype(int)
    pryoe_df.set_index(['returnerId'], inplace=True)
    players_df.set_index(['returnerId'], inplace=True)
    pryoe_df = pryoe_df.join(players_df, on='returnerId', how='left')
    pryoe_df = pryoe_df.drop(['index'], axis=1)
    return pryoe_df

def l2_normalize(col):
    col = col.to_numpy()
    col = col.reshape(-1,1)
    col = normalize(col, axis=0)
    col = col.flatten()
    return pd.Series(col)

def evaluate_players(params_df, returners_df, return_model, yardage_model):
    params_df = params_df.copy()
    pyroe_df = pd.DataFrame(columns=['returnerId', 'PRYOE', 'PRYOE_AVG', 'returnerAvg', 'numReturns'])
    headers = ['gameId', 'playId', 'event', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'isReturn', 'kickReturnYardage', 'possessionTeam', 'season']
    features = [col for col in params_df.columns if col not in headers]
    params_df.fillna(0.0, inplace=True)
    for feature in features:
        params_df.loc[:,feature] = l2_normalize(params_df[feature])
    for returner in returners_df.iterrows():
        returnerId = returner[0]
        if not ';' in str(returnerId):
            returns = params_df.query('returnerId == @returnerId') 
            returner_pryoe = 0
            returner_yds = []
            for _return in returns.iterrows():
                yret = _return[1]['isReturn']
                yyrd = _return[1]['kickReturnYardage']
                X = _return[1].drop(headers)
                predReturn = return_model.predict_proba([X])
                if predReturn[0][1] > 0.6:
                    predYds = yardage_model.predict([X])[0]
                    pryoe = yyrd - predYds
                    returner_pryoe += pryoe
                    returner_yds.append(yyrd)
            try:
                returnerAvg = sum(returner_yds) / len(returner_yds)
                pryoeAvg = returner_pryoe / len(returner_yds)
            except:
                returnerAvg = 0
                pryoeAvg = 0
            row = {'returnerId': returnerId, 'PRYOE': returner_pryoe, 'PRYOE_AVG': pryoeAvg, 'returnerAvg': returnerAvg, 'numReturns': len(returner_yds)}
            pyroe_df = pyroe_df.append(row, ignore_index=True)
    pyroe_df.sort_values(by='PRYOE_AVG', axis=0, inplace=True, ascending=False, na_position='last', ignore_index=True)
    return pyroe_df

return_model = fit_logistic_model(params_df)
yardage_model = fit_linear_model(params_df)
pryoe_df = evaluate_players(params_df, returners_df, return_model, yardage_model)
pryoe_df = format_player_results(pryoe_df, players_df)
pryoe_df = pryoe_df.query('numReturns > 5')
pryoe_df.head(10)

These results seem to pass the eye test, as the majority of these players are held in high esteem for their return abilities.  Now that we have established a metric for quantifying exceptional punt return performance, hopefully it can be used to identify value in players that had previously gone unnoticed.  

# Units
Now that we have identified the best returners, we can take a look at which punt coverage units have given up the least PRYOE.  Unlike punt returners, the individual players that make up a punt coverage unit change year to year, so in order to have fair comparisons we must split out teams by season.  

In [ ]:
def build_units_df(games_df, plays_df):
    units_df = games_df[['gameId', 'season']]
    plays_df = plays_df[['gameId', 'possessionTeam']]
    
    units_df = units_df.reset_index(drop=True)
    plays_df = plays_df.reset_index(drop=True)
    
    units_df = units_df.set_index('gameId')
    plays_df = plays_df.set_index('gameId')

    units_df = plays_df.join(units_df, on='gameId', how='inner', rsuffix='_r')
    units_df = units_df.drop_duplicates()
    return units_df

def evaluate_units(params_df, return_model, yardage_model):
    params_df = params_df.copy()
    unit_pyroe_df = pd.DataFrame(columns=['Team', 'season', 'PRYOE', 'PRYOE_AVG', 'returnAvg', 'numReturns'])
    units_df = build_units_df(games_df, plays_df)
    headers = ['gameId', 'playId', 'event', 'specialTeamsPlayType', 'kickerId', 'returnerId', 'isReturn', 'kickReturnYardage', 'possessionTeam', 'season']
    features = [col for col in params_df.columns if col not in headers]
    params_df.fillna(0.0, inplace=True)
    for feature in features:
        params_df[feature] = l2_normalize(params_df[feature])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for unit in units_df.iterrows():
            unit_team = unit[1].possessionTeam
            unit_season = unit[1].season
            returns = params_df.query('possessionTeam == @unit_team & season == @unit_season')
            unit_pryoe = 0
            unit_yds = []
            for _return in returns.iterrows():
                yyrd = _return[1]['kickReturnYardage']
                X = _return[1].drop(headers)
                predReturn = return_model.predict_proba([X])
                if predReturn[0][1] > 0.6:
                        predYds = yardage_model.predict([X])[0]
                        pryoe = yyrd - predYds
                        unit_pryoe += pryoe
                        unit_yds.append(yyrd)
            try:
                unitAvg = sum(unit_yds) / len(unit_yds)
                pryoeAvg = unit_pryoe / len(unit_yds)
            except:
                unitAvg = 0
                pryoeAvg = 0
            row = {'Team': unit_team, 'season': unit_season,'PRYOE': unit_pryoe, 'PRYOE_AVG': pryoeAvg, 'returnAvg': unitAvg, 'numReturns': len(unit_yds)}
            unit_pyroe_df = unit_pyroe_df.append(row, ignore_index=True)
    unit_pyroe_df.sort_values(by='PRYOE_AVG', axis=0, inplace=True, ascending=True, na_position='last', ignore_index=True)
    unit_pyroe_df.head()
    return unit_pyroe_df
units_df = evaluate_units(params_df, return_model, yardage_model)
units_df.head(10)

Something that immediately jumps out from the results is that several top units are from the 2018 season.  This could happen for a variety of reasons.  Perhaps teams have decided to invest more in quality punt returners, or invest less in punt coverage units over the last few years.  It is also worth noting that a low number of total returns in this case does not necessarily indicate a low number of punts, as punts that went out of bounds or for touchbacks were omitted from the dataset.  

# Further Applications
## Players
Now that we have a metric for evaluating a players **YoE**, we can use it to further analyze players in a variety of ways.  Potential followup projects include:
* Running a clustering algorithm on **PRYOE** and its features to see if we can narrow down what physical/athletic characteristics make a successful return.
* Run this same algorithm, but partitioned by year, then build a time-series forecasting function to predict future performance of both established players and rookies/prospects.
* Given past college tracking data, build a function that predicts NFL **PRYOE** based on CFB **PRYOE**, opening a new avenue of draft analysis.

## Units
Of course, potential applications reach further than just player analysis.  Since a punt coverage unit is made up of 11+ players, we can examine performance in several ways.  Using our **PRYOEA** metric, we could:
* Evaluate Special Teams coordinator performance across multiple years and teams.  Teams like the Jaguars and Seahawks seem to have consistently good **PRYOEA** numbers, what do their ST coordinators do better then most?
* Evaluate Punt Coverage Unit performance independent of coordinator.  For example, the Packers have had multiple ST coordinators, yet their scores are consistently bad.  What can we learn from their strategy and unit composition?
* Run a clustering algorithm on unit composition.  What player archetypes do successful units have? Are there certain characteristics that are more important than others?
* Chart **PRYOEA** numbers for players across teams and years.  Are there players that are more impactful to their coverage unit that others? And if so, what do they have in common?

# Conclusion
Although there is much that we can learn using the **PRYOE** metric we have developed, the number is only the beginning.  Quantifying these plays unlocks a whole treasure chest of secondary applications and analysis.  Although this model is by no means perfect, it is my hope that it can inspire and inform future player and strategy analysis and continue to advance the game.  